<a href="https://colab.research.google.com/github/redish0317/cp1/blob/main/cp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import time
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)

def crawl_id(limit=100,offset=0, tag_type=518):
    url = 'http://www.wanted.jobs/api/v4/jobs?'

    params ={1656232918453:'',    #사용자번호?
    'country': 'all',
    'tag_type_ids': tag_type, #개발직
    'job_sort': 'job.latest_order',    #최신순 정렬
    'locations': all,
    'years': -1,    #경력 이상
    'years': -1,    #경력 이하    경력상관없이 검색하려면 -1
    'limit': limit,    #한 번에 조회 가능한 수 (최대100)
    'offset': offset}    #조회할 게시물의 첫 index        ex) limit=100 offset=10  => 10번게시물부터 110번게시물까지 크롤링

    #서버에 url과 쿼리로 요청
    r = requests.get(url,
                     params = params)
    #요청한 데이터 json포멧으로 변환
    r = r.json()
    #json포멧 데이터중 id컬럼만 추출
    id_list = [i['id'] for i in r['data']]
    return id_list

def return_id_list():
    '''
    0번째 게시물부터 100개씩 크롤링 while true
    오류발생! => ex) 총 게시물이 321개인데 300개 크롤링 후 다음100개를 크롤링하려했기때문
    따라서 재귀호출을 통해 크롤링 수를 100개씩 -> 오류발생! -> 10개씩 -> 오류발생! -> 1개씩 크롤링하는 함수구현
    만약 게시물이 321개라면 300개 크롤링 -> 20개 크롤링 - 1개 크롤링 return
    '''
    id_list=[]
    def crawl_all_id(limit=100,offset=0):
        try:
            while True:
                id_list.extend(crawl_id(limit,offset))
                offset+=limit

        except:
            if limit != 1:
                return crawl_all_id(limit/10,offset)
    crawl_all_id()
    return id_list


def crawl_job(id_list):
    df_list = []
    
    for id in id_list:
        url = f'https://www.wanted.jobs/api/v4/jobs/{id}?1656259528432'
        r = requests.get(url)
        r = r.json()['job']

        #1개의 게시물 크롤링할때마다 데이터프레임에 append 또는 concat하는것보다
        #list에 append하고 마지막에 한번에 concat하는게 속도가 더 빠르다고 함
        df_list.append(pd.json_normalize(r))
        
    df = pd.concat(df_list, ignore_index=True)
    #단일 게시물을 크롤링해서 concat하다보니 index가 모두 0이므로 reset_index
    return df


def engineering(df):
    #사용하지 않을 컬럼 drop
    drop_col = ['is_crossboarder','like_count','is_like','score','company_images','status','is_bookmark','is_company_follow',
            'compare_country','matching_score','short_link','address.geo_location.n_location.address','address.geo_location.location.lat','address.geo_location.location.lng','address.geo_location.viewport.northeast.lat',
            'address.geo_location.viewport.northeast.lng','address.geo_location.viewport.southwest.lat','address.geo_location.viewport.southwest.lng','address.geo_location.bounds','detail.intro','detail.benefits','company.id','company.application_response_stats.level',
            'company.application_response_stats.delayed_count', 'company.application_response_stats.remained_count', 'company.application_response_stats.type', 'company.name', 'logo_img.origin', 'logo_img.thumb', 'title_img.origin', 'title_img.thumb', 'reward.formatted_total', 'reward.formatted_recommender', 'reward.formatted_recommendee', 'address.geo_location.bounds.northeast.lat', 'address.geo_location.bounds.northeast.lng', 'address.geo_location.bounds.southwest.lat', 'address.geo_location.bounds.southwest.lng']
    df = df.drop(drop_col,axis=1).iloc[:,1:]

    try:
        #skill_tags 비어있는 행 제거
        df = df.loc[df['skill_tags'].apply(lambda x :  x !=[])]
        #skill_tags 문자열에서 키워드만 추출
        df['skill_tags'] = df['skill_tags'].apply(lambda x : [list(i.values())[0] for i  in x])
        #company_tags 문자열에서 키워드만 추출
        df['company_tags'] = df['company_tags'].apply(lambda x : [i['title'] for i  in x])
        #category_tags 직무id만 추출
        df['category_tags'] = df['category_tags'].apply(lambda x : [i['id'] for i  in x])

    #크롤링시 가끔 dict list가 문자열형태로 존재하는 경우가 발생 ex) '[a,b,c,d]', '{a:1,b:2,c:3}' 이런경우 문자열로접근
    except:
        #skill_tags 비어있는 행 제거
        df = df.loc[df['skill_tags'].apply(lambda x :  x !='[]')]
        #skill_tags 문자열에서 키워드만 추출
        df['skill_tags'] = df['skill_tags'].apply(lambda x : [i.split(',')[0][2:-1] for i in x.split(':') if 'id' in i])
        #company_tags 문자열에서 키워드만 추출
        df['company_tags'] = df['company_tags'].apply(lambda x : [i.split(',')[0][2:-1] for i in x.split(':') if 'id' in i][:-1])
        #category_tags 직무id만 추출
        df['category_tags'] = df['category_tags'].apply(lambda x : [re.sub(r'[^0-9]', '', v) for i,v in enumerate(x.split(':')) if i % 2 == 0][1:])
    return df


In [17]:
start = time.time()
id_list = return_id_list()
end = time.time()
print(end-start)

start = time.time()
df = crawl_job(id_list)
end = time.time()
print(end-start)

23.346116542816162
346.7998173236847


In [ ]:
start = time.time()

tag_type_list = [1024, 1025, 10231, 1634] # 데이터사이언티스트, 빅데이터엔지니어, DBA, 머신러닝엔지니어
id_list = []
for tag in tag_type_list:
    id_list.extend(return_id_list(tag))

end = time.time()

print(end-start)

In [1]:
len(id_list)

NameError: ignored

In [25]:
len(a)

559

In [19]:
df = engineering(df)
df.head()

,due_time,skill_tags,company_tags,position,category_tags,address.country,address.full_location,address.geo_location.n_location.lat,address.geo_location.n_location.lng,address.geo_location.location_type,address.location,address.country_code,detail.requirements,detail.main_tasks,detail.preferred_points,company.industry_name,company.application_response_stats.avg_rate,company.application_response_stats.avg_day,address.geo_location
0,None,"[Pytorch, Tensorflow, Python, AWS]","[Startup, Snacks, AI, Work From Home, Professi...",기계학습/인공지능 연구개발,"[1024, 1025, 1634]",Korea,"대전광역시 유성구 대학로 157, 디1동 3층 301호(궁동, 디브릿지)",36.360625,127.350838,ROOFTOP,Daejeon,kr,＜ 경력 ＞\n• 생체신호 분석에 대한 이해 및 관련 경험 보유\n• 인공지능(머신...,＜ 연구 분야 ＞\n• 기계학습/인공지능을 이용한 뇌영상(EEG/fMRI/fNIRS...,"• 컴퓨터공학, 생명공학, 생명과학 등 관련 분야 전공자\n• 인공지능, 뇌과학, ...","Professional, Scientific",75.0,13,NaN
1,None,"[Python, Tableau]","[Rapid Company Growth, 51~300 employees, Compa...",NLP 머신러닝 엔지니어 연구원,[1634],Korea,"서울특별시 강남구 논현로 72길 23, 2층 (역삼동770, 송하빌딩)",37.496353,127.04065,ROOFTOP,Seoul,kr,"• 컴퓨터공학, 머신러닝 관련 분야 석사 이상의 학위를 취득하신 분\n• 딥러닝 프...",• 한국어 대화 모델 연구 및 개발\n• 한국어 자연어 모델링\n• 문제 생성 작업,• 비정형 텍스트 데이터의 모델링 경험이 있으신 분\n• NLP 오픈소스의 활용 및...,Other services,87.3,2,NaN
2,None,"[Python, Tableau]","[Rapid Company Growth, 51~300 employees, Compa...",TTS 머신러닝 엔지니어 연구원,[1634],Korea,"서울특별시 강남구 논현로 72길 23, 2층 (역삼동770, 송하빌딩)",37.496353,127.04065,ROOFTOP,Seoul,kr,"• 컴퓨터공학, 머신러닝 관련 분야 석사 이상의 학위를 취득하신 분 또는 3년 이상...",• Acoustic Model 연구 및 개발\n• Neural Vocoder 연구 ...,• 음성합성용 음성 DB구축/가공 경험을 보유하신 분\n• E2E 음성합성 오픈소스...,Other services,87.3,2,NaN
3,None,"[Pytorch, Tensorflow, Python, EDA]","[Salary Top 1%, Rapid Company Growth, 51~300 e...",Machine Learning Engineer,[1634],Korea,"서울특별시 서초구 강남대로 343 , 12층",37.493519,127.029259,ROOFTOP,Seoul,kr,"- ML/DS 관련 경력이 2년 이상이신 분\n- 자료구조, 알고리즘 등에 대한 기...","#ML Solution팀은 이런 일을 합니다.\n- 제조, 에너지 분야에서 발생하는...",- 실제 문제에 대해 탐색적 데이터 분석(EDA) 경험이 있으신 분\n- AI 모델...,"IT, Media",97.1,1,NaN
4,2022-07-10,"[Android, AngularJS, Firebase, Flask, Google C...","[Rapid Company Growth, 51~300 employees, Compa...",[데이터팀] NLP 엔지니어,"[655, 1024, 1634]",Korea,서울특별시 강남구 테헤란로 431(삼성동) 저스트코 타워 14층,37.506533,127.054287,ROOFTOP,Seoul,kr,이런 분을 기다려요. \n________________________________...,자란다의 데이터팀은?\n_________________________________...,이런 경험이 있다면 더욱 감사해요! \n________________________...,"IT, Media",91.1,2,NaN


TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[1634]                70
[1024, 1025, 1634]    50
[655, 1024, 1634]     37
[655, 1025, 1634]     18
[899, 1634, 10110]    16
                      ..
[898, 1634, 10110]     1
[658, 900, 1634]       1
[899, 939, 1634]       1
[674, 1634, 10110]     1
[660, 1025, 1634]      1
Name: category_tags, Length: 71, dtype: int64

In [ ]:
df.to_csv('wanted.csv')